# XTTS v2 - Clonage Vocal Zero-Shot

**Module :** 02-Audio-Advanced  
**Niveau :** Intermediaire  
**Technologies :** Coqui XTTS v2, ~6 GB VRAM  
**Duree estimee :** 45 minutes  

## Objectifs d'Apprentissage

- [ ] Installer et charger le modele XTTS v2 depuis Coqui TTS
- [ ] Comprendre le clonage vocal zero-shot (pas de fine-tuning)
- [ ] Cloner une voix a partir d'un clip audio de reference (~6s)
- [ ] Exploiter le support multilingue (17 langues dont le francais)
- [ ] Analyser la qualite du clonage (similarite speaker avec resemblyzer)
- [ ] Comparer XTTS v2 avec Chatterbox et OpenAI TTS
- [ ] Comprendre les enjeux ethiques du clonage vocal

## Prerequis

- GPU NVIDIA avec au moins 6 GB VRAM
- `pip install TTS`
- Notebook 02-1 recommande (pour comparaison avec Chatterbox)

**Navigation :** [<< 02-1](02-1-Chatterbox-TTS.ipynb) | [Index](../README.md) | [Suivant >>](02-3-MusicGen-Generation.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres XTTS
model_name = "tts_models/multilingual/multi-dataset/xtts_v2"  # Modele XTTS v2
device = "cuda"                     # "cuda" ou "cpu"
language = "fr"                     # Langue par defaut

# Configuration
generate_audio = True              # Generer les fichiers audio
save_results = True                # Sauvegarder les fichiers generes
test_multilingual = True           # Tester plusieurs langues
analyze_similarity = True          # Analyser la similarite vocale

In [2]:
# Setup environnement et imports
import os
import sys
import json
import time
import gc
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import logging

import numpy as np
import soundfile as sf
from IPython.display import Audio, display, HTML

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.audio_helpers import play_audio, save_audio
        print("Helpers audio importes")
    except ImportError:
        print("Helpers audio non disponibles - mode autonome")

# Repertoires
OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'xtts'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('xtts_cloning')

# Verification GPU
gpu_available = False
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        gpu_name = torch.cuda.get_device_name(0)
        gpu_vram = torch.cuda.get_device_properties(0).total_mem / (1024**3)
        print(f"GPU : {gpu_name} ({gpu_vram:.1f} GB VRAM)")
    else:
        print("GPU non disponible - XTTS fonctionne aussi sur CPU (lentement)")
        if device == "cuda":
            device = "cpu"
            print("Fallback vers CPU")
except ImportError:
    print("torch non installe")
    device = "cpu"

print(f"\nXTTS v2 - Clonage Vocal Zero-Shot")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}, Device : {device}")
print(f"Langue : {language}")
print(f"Sortie : {OUTPUT_DIR}")

Helpers audio importes


GPU non disponible - XTTS fonctionne aussi sur CPU (lentement)
Fallback vers CPU

XTTS v2 - Clonage Vocal Zero-Shot
Date : 2026-02-18 10:33:16
Mode : interactive, Device : cpu
Langue : fr
Sortie : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\xtts


In [3]:
# Chargement .env
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve")

# XTTS ne necessite pas de cle API, mais HuggingFace peut etre utile
hf_token = os.getenv('HUGGINGFACE_TOKEN') or os.getenv('HF_TOKEN')
if hf_token:
    print(f"Token HuggingFace disponible")
else:
    print(f"Token HuggingFace non disponible (telechargement public uniquement)")

Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env
Token HuggingFace non disponible (telechargement public uniquement)


## Dependances GPU OptionnellesCe notebook utilise des modeles GPU optionnels. Pour les activer:Sans ces dependances, le notebook s'executera en mode API uniquement.

## Section 1 : Presentation de XTTS v2

XTTS v2 (Cross-lingual Text-To-Speech) est le modele phare de Coqui AI. Il permet le clonage vocal zero-shot dans 17 langues a partir d'un simple clip audio de reference.

### Langues supportees

| Code | Langue | Code | Langue |
|------|--------|------|--------|
| `en` | Anglais | `fr` | Francais |
| `es` | Espagnol | `de` | Allemand |
| `it` | Italien | `pt` | Portugais |
| `pl` | Polonais | `tr` | Turc |
| `ru` | Russe | `nl` | Neerlandais |
| `cs` | Tcheque | `ar` | Arabe |
| `zh` | Chinois | `ja` | Japonais |
| `ko` | Coreen | `hu` | Hongrois |
| `hi` | Hindi | | |

### Comparaison avec les autres modeles

| Aspect | XTTS v2 | Chatterbox | OpenAI TTS |
|--------|---------|-----------|------------|
| Clonage vocal | Zero-shot (6s) | Voice conditioning | Non |
| Langues | 17 | Anglais | Multilingue |
| VRAM | ~6 GB | ~8 GB | N/A |
| Licence | CPML (non-commercial) | MIT | Proprietaire |
| Qualite | Tres bonne | Bonne (expressif) | Excellente |

In [4]:
# Chargement du modele XTTS v2
print("CHARGEMENT DU MODELE XTTS V2")
print("=" * 45)

xtts_loaded = False

try:
    from TTS.api import TTS

    print(f"Chargement {model_name}...")
    print(f"(Premier lancement : telechargement du modele ~1.8 GB)")
    start_time = time.time()

    tts = TTS(model_name=model_name).to(device)
    load_time = time.time() - start_time
    xtts_loaded = True

    print(f"Modele charge en {load_time:.1f}s")
    print(f"Device : {device}")

    if gpu_available:
        vram_used = torch.cuda.memory_allocated(0) / (1024**3)
        print(f"VRAM utilisee : {vram_used:.2f} GB")

except ImportError:
    print("TTS non installe")
    print("Installation : pip install TTS")
except Exception as e:
    print(f"Erreur lors du chargement : {type(e).__name__} - {str(e)[:200]}")

CHARGEMENT DU MODELE XTTS V2
TTS non installe
Installation : pip install TTS


## Section 2 : Clonage vocal zero-shot

Le clonage zero-shot signifie qu'aucun entrainement supplementaire n'est necessaire. Le modele utilise directement un clip audio (~6s) pour capturer le timbre du locuteur.

### Pipeline de clonage

| Etape | Description | Detail |
|-------|-------------|--------|
| 1 | Preparation du clip | WAV, 16-24kHz, ~6s, sans bruit |
| 2 | Extraction d'embeddings | Le modele encode les caracteristiques vocales |
| 3 | Generation conditionnee | Le texte est synthetise avec le timbre capture |
| 4 | Decodage audio | Conversion des tokens en signal audio |

In [5]:
# Clonage vocal zero-shot
print("CLONAGE VOCAL ZERO-SHOT")
print("=" * 45)

clone_text_fr = (
    "Bonjour, je suis une voix clonee par le modele XTTS version deux. "
    "Cette technologie permet de reproduire le timbre d'une voix "
    "a partir d'un simple enregistrement de quelques secondes."
)

if xtts_loaded and generate_audio:
    # Creer un clip de reference synthetique pour la demonstration
    # En production, on utiliserait un vrai enregistrement vocal
    print("Creation d'un clip de reference...")

    ref_path = OUTPUT_DIR / "reference_speaker.wav"

    # Generer un premier clip comme reference
    tts.tts_to_file(
        text="This is a sample reference voice clip for zero-shot voice cloning.",
        file_path=str(ref_path),
        language="en"
    )
    print(f"Clip de reference cree : {ref_path.name}")

    ref_data, ref_sr = sf.read(str(ref_path))
    print(f"  Duree : {len(ref_data)/ref_sr:.1f}s, Sample rate : {ref_sr} Hz")
    display(Audio(data=ref_data, rate=ref_sr))

    # Clonage avec le clip de reference
    print(f"\nGeneration avec clonage vocal (langue : {language})...")
    start_time = time.time()

    output_path = OUTPUT_DIR / "cloned_voice_fr.wav"
    tts.tts_to_file(
        text=clone_text_fr,
        file_path=str(output_path),
        speaker_wav=str(ref_path),
        language=language
    )

    gen_time = time.time() - start_time

    cloned_data, cloned_sr = sf.read(str(output_path))
    duration = len(cloned_data) / cloned_sr

    print(f"  Duree : {duration:.1f}s")
    print(f"  Temps de generation : {gen_time:.2f}s")
    print(f"  Ratio temps reel : {duration / gen_time:.1f}x")
    display(Audio(data=cloned_data, rate=cloned_sr))
else:
    print("Modele non charge ou generation desactivee")

CLONAGE VOCAL ZERO-SHOT
Modele non charge ou generation desactivee


### Interpretation : Clonage vocal

| Aspect | Valeur typique | Signification |
|--------|---------------|---------------|
| Ratio temps reel | 2-5x (GPU) | Plus lent que Kokoro mais acceptable |
| Similarite vocale | Elevee | Le timbre est bien capture en zero-shot |
| Qualite francais | Bonne | XTTS supporte nativement le francais |

**Points cles** :
1. Le clonage zero-shot ne necessite aucun fine-tuning
2. La qualite du clip de reference impacte directement le resultat
3. Le modele preserve le timbre meme en changeant de langue

## Section 3 : Support multilingue

L'un des atouts majeurs de XTTS v2 est le support cross-lingual : on peut cloner une voix dans une langue et generer de la parole dans une autre.

| Scenario | Description |
|----------|-------------|
| Meme langue | Clip en francais, generation en francais |
| Cross-lingual | Clip en anglais, generation en francais |
| Multi-output | Un meme clip, generation dans plusieurs langues |

In [6]:
# Test multilingue
print("TEST MULTILINGUE")
print("=" * 45)

multilingual_texts = {
    "fr": "Bonjour, ceci est un test de synthese vocale en francais.",
    "en": "Hello, this is a test of speech synthesis in English.",
    "es": "Hola, esta es una prueba de sintesis de voz en espanol.",
    "de": "Hallo, dies ist ein Test der Sprachsynthese auf Deutsch.",
}

multilingual_results = {}

if xtts_loaded and generate_audio and test_multilingual:
    ref_path = OUTPUT_DIR / "reference_speaker.wav"

    if not ref_path.exists():
        print("Clip de reference non disponible - creation...")
        tts.tts_to_file(
            text="This is a reference clip for multilingual testing.",
            file_path=str(ref_path),
            language="en"
        )

    for lang_code, text in multilingual_texts.items():
        print(f"\nLangue : {lang_code}")
        print(f"  Texte : {text}")

        try:
            start_time = time.time()
            out_path = OUTPUT_DIR / f"multilingual_{lang_code}.wav"

            tts.tts_to_file(
                text=text,
                file_path=str(out_path),
                speaker_wav=str(ref_path),
                language=lang_code
            )

            gen_time = time.time() - start_time
            audio_data, sr = sf.read(str(out_path))
            duration = len(audio_data) / sr

            multilingual_results[lang_code] = {
                "duration": duration,
                "gen_time": gen_time,
                "text_len": len(text)
            }

            print(f"  Duree : {duration:.1f}s | Temps : {gen_time:.2f}s")
            display(Audio(data=audio_data, rate=sr))

        except Exception as e:
            print(f"  Erreur : {str(e)[:80]}")

    # Tableau recapitulatif
    if multilingual_results:
        print(f"\nRecapitulatif multilingue :")
        print(f"{'Langue':<10} {'Duree (s)':<12} {'Temps gen (s)':<15} {'Chars':<8}")
        print("-" * 45)
        for lang, data in multilingual_results.items():
            print(f"{lang:<10} {data['duration']:<12.1f} {data['gen_time']:<15.2f} {data['text_len']:<8}")
else:
    print("Test multilingue desactive ou modele non charge")

TEST MULTILINGUE
Test multilingue desactive ou modele non charge


### Interpretation : Support multilingue

| Langue | Qualite | Observation |
|--------|---------|-------------|
| Anglais | Excellente | Langue d'entrainement principale |
| Francais | Tres bonne | Bonne prononciation et prosodie |
| Espagnol | Tres bonne | Accent naturel |
| Allemand | Bonne | Phonemes complexes bien geres |

**Points cles** :
1. Le timbre est preserve d'une langue a l'autre (cross-lingual)
2. La qualite varie selon la representation de la langue dans les donnees d'entrainement
3. Les langues romanes (fr, es, it, pt) donnent generalement de tres bons resultats

## Section 4 : Analyse de la qualite du clonage

Pour evaluer objectivement la qualite du clonage vocal, on peut utiliser des metriques de similarite entre le locuteur de reference et la voix generee.

### Metriques de similarite

| Metrique | Description | Plage |
|----------|-------------|-------|
| Cosine similarity (embeddings) | Distance entre embeddings de speaker | 0-1 (1 = identique) |
| MOS (Mean Opinion Score) | Evaluation perceptuelle humaine | 1-5 (5 = excellent) |
| Speaker Error Rate (SER) | Taux d'erreur d'identification | 0-100% (0 = parfait) |

In [7]:
# Analyse de similarite vocale
print("ANALYSE DE SIMILARITE VOCALE")
print("=" * 45)

if xtts_loaded and generate_audio and analyze_similarity:
    similarity_available = False

    try:
        from resemblyzer import VoiceEncoder, preprocess_wav
        encoder = VoiceEncoder()
        similarity_available = True
        print("Resemblyzer charge pour l'analyse de similarite")
    except ImportError:
        print("resemblyzer non installe (pip install resemblyzer)")
        print("Analyse de similarite par correlation simple")

    # Charger les fichiers audio
    ref_path = OUTPUT_DIR / "reference_speaker.wav"
    clone_path = OUTPUT_DIR / "cloned_voice_fr.wav"

    if ref_path.exists() and clone_path.exists():
        ref_audio, ref_sr = sf.read(str(ref_path))
        clone_audio, clone_sr = sf.read(str(clone_path))

        if similarity_available:
            # Analyse avec resemblyzer
            ref_processed = preprocess_wav(ref_audio, source_sr=ref_sr)
            clone_processed = preprocess_wav(clone_audio, source_sr=clone_sr)

            ref_embed = encoder.embed_utterance(ref_processed)
            clone_embed = encoder.embed_utterance(clone_processed)

            # Similarite cosinus
            cosine_sim = np.dot(ref_embed, clone_embed) / (
                np.linalg.norm(ref_embed) * np.linalg.norm(clone_embed)
            )

            print(f"\nResultats de similarite :")
            print(f"  Similarite cosinus : {cosine_sim:.4f}")
            print(f"  Interpretation : ", end="")
            if cosine_sim > 0.85:
                print("Excellent - voix tres similaire")
            elif cosine_sim > 0.75:
                print("Bon - voix reconnaissable")
            elif cosine_sim > 0.65:
                print("Moyen - timbre partiellement capture")
            else:
                print("Faible - voix differente")
        else:
            # Analyse simplifiee sans resemblyzer
            min_len = min(len(ref_audio), len(clone_audio))
            if min_len > 0:
                # Correlation sur les enveloppes spectrales
                from scipy.signal import hilbert
                ref_env = np.abs(hilbert(ref_audio[:min_len]))
                clone_env = np.abs(hilbert(clone_audio[:min_len]))
                corr = np.corrcoef(ref_env, clone_env)[0, 1]
                print(f"  Correlation d'enveloppe : {corr:.4f}")

        # Statistiques comparatives
        print(f"\nStatistiques audio :")
        print(f"  {'Metrique':<25} {'Reference':<15} {'Clone':<15}")
        print(f"  {'-'*55}")
        print(f"  {'Duree (s)':<25} {len(ref_audio)/ref_sr:<15.1f} {len(clone_audio)/clone_sr:<15.1f}")
        print(f"  {'Sample rate (Hz)':<25} {ref_sr:<15} {clone_sr:<15}")
        print(f"  {'RMS energy':<25} {np.sqrt(np.mean(ref_audio**2)):<15.4f} {np.sqrt(np.mean(clone_audio**2)):<15.4f}")
        print(f"  {'Peak amplitude':<25} {np.max(np.abs(ref_audio)):<15.4f} {np.max(np.abs(clone_audio)):<15.4f}")
    else:
        print("Fichiers audio non disponibles pour l'analyse")
else:
    print("Analyse de similarite desactivee ou modele non charge")

ANALYSE DE SIMILARITE VOCALE
Analyse de similarite desactivee ou modele non charge


### Interpretation : Qualite du clonage

| Metrique | Valeur typique XTTS | Signification |
|----------|-------------------|---------------|
| Cosine similarity | 0.75-0.90 | Bon a excellent clonage |
| RMS energy | Variable | Normalise par le modele |
| Peak amplitude | ~0.8-1.0 | Audio bien normalise |

**Points cles** :
1. Une similarite > 0.85 indique un clonage de tres bonne qualite
2. La qualite du clip de reference est le facteur le plus important
3. Le clonage cross-lingual peut reduire legerement la similarite

> **Note ethique** : Les systemes de clonage vocal soulevent des enjeux majeurs. Le consentement du locuteur est indispensable. Coqui impose une licence non-commerciale (CPML) pour limiter les usages abusifs.

In [8]:
# Mode interactif - Clonage personnalise
if notebook_mode == "interactive" and not skip_widgets:
    print("MODE INTERACTIF - CLONAGE PERSONNALISE")
    print("=" * 50)
    print("\nEntrez un texte a synthetiser avec la voix clonee :")
    print("(Laissez vide pour passer a la suite)")

    try:
        user_text = input("\nVotre texte : ")

        if user_text.strip() and xtts_loaded:
            user_lang = input(f"Langue [{language}] (fr/en/es/de/it/pt) : ").strip() or language

            ref_path = OUTPUT_DIR / "reference_speaker.wav"
            if ref_path.exists():
                print(f"\nGeneration avec clonage vocal (langue : {user_lang})...")
                start_time = time.time()

                out_path = OUTPUT_DIR / f"custom_clone_{user_lang}.wav"
                tts.tts_to_file(
                    text=user_text,
                    file_path=str(out_path),
                    speaker_wav=str(ref_path),
                    language=user_lang
                )

                gen_time = time.time() - start_time
                audio_data, sr = sf.read(str(out_path))
                print(f"Duree : {len(audio_data)/sr:.1f}s | Temps : {gen_time:.2f}s")
                display(Audio(data=audio_data, rate=sr))
            else:
                print("Clip de reference non disponible")
        else:
            print("Mode interactif ignore")

    except (KeyboardInterrupt, EOFError):
        print("Mode interactif interrompu")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("Mode interactif non disponible (execution automatisee)")
        else:
            print(f"Erreur : {error_type} - {str(e)[:100]}")
else:
    print("Mode batch - Interface interactive desactivee")

MODE INTERACTIF - CLONAGE PERSONNALISE

Entrez un texte a synthetiser avec la voix clonee :
(Laissez vide pour passer a la suite)
Mode interactif non disponible (execution automatisee)


## Considerations ethiques et bonnes pratiques

### Enjeux du clonage vocal

| Risque | Description | Mitigation |
|--------|-------------|------------|
| Usurpation d'identite | Utiliser la voix d'une personne sans autorisation | Consentement ecrit obligatoire |
| Deepfakes audio | Creer de faux enregistrements | Watermarking, detection |
| Fraude | Imiter une voix pour tromper | Authentification multi-facteurs |
| Desinformation | Creer de faux discours | Legislation, detection |

### Bonnes pratiques

| Pratique | Description |
|----------|-------------|
| Consentement | Toujours obtenir l'accord ecrit du locuteur |
| Watermarking | Ajouter un filigrane audio aux voix synthetiques |
| Documentation | Indiquer clairement qu'un audio est synthetique |
| Licence | Respecter la licence CPML (non-commercial) de XTTS |

In [9]:
# Statistiques de session et prochaines etapes
print("STATISTIQUES DE SESSION")
print("=" * 45)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Modele : {model_name}")
print(f"Device : {device}")
print(f"Langue par defaut : {language}")
print(f"Modele charge : {'Oui' if xtts_loaded else 'Non'}")

if gpu_available:
    vram_current = torch.cuda.memory_allocated(0) / (1024**3)
    print(f"VRAM utilisee : {vram_current:.2f} GB")

if save_results:
    saved = list(OUTPUT_DIR.glob('*'))
    total_size = sum(f.stat().st_size for f in saved) / (1024*1024)
    print(f"Fichiers sauvegardes : {len(saved)} ({total_size:.1f} MB) dans {OUTPUT_DIR}")

# Liberation memoire
if xtts_loaded:
    print(f"\nLiberation du modele...")
    del tts
    gc.collect()
    if gpu_available:
        torch.cuda.empty_cache()
    print(f"Memoire liberee")

print(f"\nPROCHAINES ETAPES")
print(f"1. Explorer la generation musicale avec MusicGen (02-3)")
print(f"2. Decouvrir la separation de sources avec Demucs (02-4)")
print(f"3. Comparer tous les modeles audio (03-1)")
print(f"4. Construire un pipeline vocal complet (03-2)")

print(f"\nNotebook XTTS Voice Cloning termine - {datetime.now().strftime('%H:%M:%S')}")

STATISTIQUES DE SESSION
Date : 2026-02-18 10:33:16
Modele : tts_models/multilingual/multi-dataset/xtts_v2
Device : cpu
Langue par defaut : fr
Modele charge : Non
Fichiers sauvegardes : 0 (0.0 MB) dans D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\xtts

PROCHAINES ETAPES
1. Explorer la generation musicale avec MusicGen (02-3)
2. Decouvrir la separation de sources avec Demucs (02-4)
3. Comparer tous les modeles audio (03-1)
4. Construire un pipeline vocal complet (03-2)

Notebook XTTS Voice Cloning termine - 10:33:16
